# QC Notebook: I/O Module Validation

This notebook validates the `starfinder.io` module functionality:
- `load_multipage_tiff`: Load single-channel TIFF files
- `load_image_stacks`: Load multi-channel stacks from a directory
- `save_stack`: Save arrays as multi-page TIFFs

**Test Dataset:** `fixtures/synthetic/mini` (1 FOV, 256x256x5, 4 channels, 4 rounds)

In [ ]:
# Setup
import sys
sys.path.insert(0, "../src/python")

import numpy as np
from pathlib import Path
import tempfile

from starfinder.io import load_multipage_tiff, load_image_stacks, save_stack
from starfinder.benchmark import measure, print_table, BenchmarkResult

# Path to mini dataset
MINI_DATASET = Path("fixtures/synthetic/mini")

print(f"Mini dataset exists: {MINI_DATASET.exists()}")
print(f"Contents: {list(MINI_DATASET.iterdir()) if MINI_DATASET.exists() else 'N/A'}")

## 1. Load Single TIFF

Test loading a single-channel TIFF file and verify the output shape and dtype.

In [ ]:
# Load ch00.tif from FOV_001/round1
tiff_path = MINI_DATASET / "FOV_001" / "round1" / "ch00.tif"
print(f"Loading: {tiff_path}")

image = load_multipage_tiff(tiff_path)

# Verify shape and dtype
print(f"\nShape: {image.shape}")
print(f"Expected: (Z, Y, X) = (5, 256, 256)")
print(f"Dtype: {image.dtype}")
print(f"Expected: uint8")

# Validation
assert image.ndim == 3, f"Expected 3D array, got {image.ndim}D"
assert image.dtype == np.uint8, f"Expected uint8, got {image.dtype}"
assert image.shape[0] == 5, f"Expected Z=5, got {image.shape[0]}"
assert image.shape[1] == 256, f"Expected Y=256, got {image.shape[1]}"
assert image.shape[2] == 256, f"Expected X=256, got {image.shape[2]}"

print("\n[PASS] Single TIFF load validation")

## 2. Load Multi-Channel Stack

Test loading all 4 channels from a round directory.

In [ ]:
# Load all 4 channels from round1
round_dir = MINI_DATASET / "FOV_001" / "round1"
channel_order = ["ch00", "ch01", "ch02", "ch03"]

print(f"Loading from: {round_dir}")
print(f"Channels: {channel_order}")

stack, metadata = load_image_stacks(round_dir, channel_order)

# Verify shape and dtype
print(f"\nShape: {stack.shape}")
print(f"Expected: (Z, Y, X, C) = (5, 256, 256, 4)")
print(f"Dtype: {stack.dtype}")
print(f"Metadata: {metadata}")

# Validation
assert stack.ndim == 4, f"Expected 4D array, got {stack.ndim}D"
assert stack.shape == (5, 256, 256, 4), f"Expected (5, 256, 256, 4), got {stack.shape}"
assert stack.dtype == np.uint8, f"Expected uint8, got {stack.dtype}"
assert metadata["cropped"] == False, "Unexpected cropping detected"

print("\n[PASS] Multi-channel stack load validation")

## 3. Save and Reload Roundtrip

Test that saving and reloading data preserves the original values.

In [ ]:
# Create test data with known values
test_data = np.random.default_rng(42).integers(0, 256, size=(5, 64, 64), dtype=np.uint8)
print(f"Test data shape: {test_data.shape}")
print(f"Test data dtype: {test_data.dtype}")
print(f"Test data range: [{test_data.min()}, {test_data.max()}]")

# Save to temporary file
with tempfile.TemporaryDirectory() as tmpdir:
    save_path = Path(tmpdir) / "test_roundtrip.tif"
    
    # Save
    save_stack(test_data, save_path)
    print(f"\nSaved to: {save_path}")
    print(f"File exists: {save_path.exists()}")
    
    # Reload
    reloaded = load_multipage_tiff(save_path, convert_uint8=False)
    print(f"Reloaded shape: {reloaded.shape}")
    print(f"Reloaded dtype: {reloaded.dtype}")
    
    # Verify equality
    assert reloaded.shape == test_data.shape, f"Shape mismatch: {reloaded.shape} vs {test_data.shape}"
    assert np.array_equal(reloaded, test_data), "Data mismatch after roundtrip"
    
print("\n[PASS] Save/reload roundtrip validation")

## 4. Benchmark Load/Save Performance

Measure the performance of I/O operations.

In [ ]:
# Benchmark single TIFF load
tiff_path = MINI_DATASET / "FOV_001" / "round1" / "ch00.tif"

_, load_time, load_mem = measure(lambda: load_multipage_tiff(tiff_path))
print(f"load_multipage_tiff:")
print(f"  Time: {load_time*1000:.2f} ms")
print(f"  Memory: {load_mem:.2f} MB")

# Benchmark multi-channel load
round_dir = MINI_DATASET / "FOV_001" / "round1"
channel_order = ["ch00", "ch01", "ch02", "ch03"]

_, stack_time, stack_mem = measure(lambda: load_image_stacks(round_dir, channel_order))
print(f"\nload_image_stacks (4 channels):")
print(f"  Time: {stack_time*1000:.2f} ms")
print(f"  Memory: {stack_mem:.2f} MB")

# Benchmark save
test_data = np.random.default_rng(42).integers(0, 256, size=(5, 256, 256), dtype=np.uint8)

with tempfile.TemporaryDirectory() as tmpdir:
    save_path = Path(tmpdir) / "benchmark_save.tif"
    
    # Without compression
    _, save_time, save_mem = measure(lambda: save_stack(test_data, save_path, compress=False))
    print(f"\nsave_stack (no compression):")
    print(f"  Time: {save_time*1000:.2f} ms")
    print(f"  Memory: {save_mem:.2f} MB")
    
    # With compression
    save_path_compressed = Path(tmpdir) / "benchmark_save_compressed.tif"
    _, save_time_c, save_mem_c = measure(lambda: save_stack(test_data, save_path_compressed, compress=True))
    print(f"\nsave_stack (with compression):")
    print(f"  Time: {save_time_c*1000:.2f} ms")
    print(f"  Memory: {save_mem_c:.2f} MB")

# Summary table
results = [
    BenchmarkResult(method="tifffile", operation="load_multipage_tiff", size=(5, 256, 256), time_seconds=load_time, memory_mb=load_mem),
    BenchmarkResult(method="tifffile", operation="load_image_stacks", size=(5, 256, 256, 4), time_seconds=stack_time, memory_mb=stack_mem),
    BenchmarkResult(method="tifffile", operation="save_stack", size=(5, 256, 256), time_seconds=save_time, memory_mb=save_mem),
    BenchmarkResult(method="tifffile+zlib", operation="save_stack", size=(5, 256, 256), time_seconds=save_time_c, memory_mb=save_mem_c),
]

print("\n--- Benchmark Summary ---")
print_table(results)

## 5. Visual Inspection with napari

Interactive visualization of loaded stacks (requires napari installation).

In [ ]:
try:
    import napari
    
    stack, _ = load_image_stacks(
        MINI_DATASET / "FOV_001" / "round1",
        ["ch00", "ch01", "ch02", "ch03"]
    )
    
    viewer = napari.Viewer()
    viewer.add_image(stack, name="FOV_001_round1", channel_axis=3)
    print("napari viewer opened. Explore the 3D stack interactively.")
    
except ImportError:
    print("napari not installed. Run: pip install napari[all]")
    print("Skipping interactive visualization.")

## Summary Checklist

| Test | Status |
|------|--------|
| load_multipage_tiff returns (Z, Y, X) shape | Verified |
| load_multipage_tiff returns uint8 dtype | Verified |
| load_image_stacks returns (Z, Y, X, C) shape | Verified |
| load_image_stacks metadata includes shape, dtype, cropped | Verified |
| save_stack/load_multipage_tiff roundtrip preserves data | Verified |
| I/O performance benchmarked | Completed |
| napari visualization (optional) | Skipped if not installed |